<a href="https://colab.research.google.com/github/mzouros/dl_gmsa/blob/main/Greek_Music_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run Colab localy

jupyter notebook \
--NotebookApp.allow_origin='https://colab.research.google.com' \
--port=8888 \
--NotebookApp.port_retries=0

In [ ]:
# Imports

import os
import glob
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time

In [12]:
# Dataset size

mp3Path = '/home/mike/Artificial Intelligence MSc/2nd Semester/4. Deep Learning/MP3'
count = 0

for root, dirs, files in os.walk(mp3Path):
  for file in files:
    if file.endswith(".mp3".casefold()):
      print(os.path.join(root, file))
      count += 1
print("Total songs found:", count)

Streaming output truncated to the last 5000 lines.
/home/mike/Artificial Intelligence MSc/2nd Semester/4. Deep Learning/MP3/Mikis Theodorakis/07 - Ante mia.mp3
/home/mike/Artificial Intelligence MSc/2nd Semester/4. Deep Learning/MP3/Mikis Theodorakis/11 - Sabbatobrado.mp3
/home/mike/Artificial Intelligence MSc/2nd Semester/4. Deep Learning/MP3/Mikis Theodorakis/8. To oneiro.mp3
/home/mike/Artificial Intelligence MSc/2nd Semester/4. Deep Learning/MP3/Mikis Theodorakis/09.NAOI STO SXHMA T' OYRANOY (orxhstra).mp3
/home/mike/Artificial Intelligence MSc/2nd Semester/4. Deep Learning/MP3/Mikis Theodorakis/11 - An8h ths petras.mp3
/home/mike/Artificial Intelligence MSc/2nd Semester/4. Deep Learning/MP3/Mikis Theodorakis/Theodorakis - Oktovris 78 - 03 - Itan kapote dyo filoi.mp3
/home/mike/Artificial Intelligence MSc/2nd Semester/4. Deep Learning/MP3/Mikis Theodorakis/09 - Xelidonaki .mp3
/home/mike/Artificial Intelligence MSc/2nd Semester/4. Deep Learning/MP3/Mikis Theodorakis/04 - Taximi.mp3

In [ ]:
# Spotify Developer Credentials

client_id = 'f1947444cbe9421e815b74a771326877'
client_secret = 'a1099b60916440aabc9a24f1583bb73e'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# Get the IDs for each playlist of the user

userId = 'mdkyyoecyz3a2cxye8qjvt91l'
playlistIds = []
offset = 0

userPlaylists1 = sp.user_playlists(userId, offset=0) # 0 - 49
userPlaylists2 = sp.user_playlists(userId, offset=50) # 50 - 99
userPlaylists3 = sp.user_playlists(userId, offset=100) # 100 - 111
for item in userPlaylists1['items']:
  playlistIds.append(item['id'])
for item in userPlaylists2['items']:
  playlistIds.append(item['id'])
for item in userPlaylists3['items']:
  playlistIds.append(item['id'])
print(playlistIds)
print(len(playlistIds))

In [ ]:
# Get the IDs for each track of each playlist of the user

user = 'mzouros'
trackIds = []

for id in playlistIds:
  playlist = sp.user_playlist(user, id)
  for item in playlist['tracks']['items']:
    track = item['track']
    trackIds.append(track['id'])

print(len(trackIds))

In [ ]:
# Get Tracks Information

for id in trackIds:
  try:
    meta = sp.track(id)
    features = sp.audio_features(id)

    # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

    # features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']

    track = [name, album, artist, release_date, length, popularity, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
    print(track)
  # We ensure that a faulty id or an empty JSON, won't crash the run  
  except JSONDecodeError:
    try:
      print("JSONDecodeError")
    except NameError:
      print("NameError")